# Tutorial 4: Saving Your Results

<div class="alert alert-block alert-info"> <b>Before we get started: </b> 
    <ul style="list-style-type: none;margin: 0;padding: 0;">
        <li>✍️ To run this notebook, you need to have Ponder installed and set up on your machine. If you have not done so already, please refer to our <a href="https://docs.ponder.io/getting_started/quickstart.html">Quickstart guide</a> to get started.</li>
        <li>📖 Otherwise, if you're just interested in browsing through the tutorial, keep reading below!</li>
    </ul>
</div>

In [1]:
import ponder; ponder.init()
import modin.pandas as pd
from google.cloud import bigquery
from google.cloud.bigquery import dbapi
from google.oauth2 import service_account
import json
bigquery_con = dbapi.Connection(bigquery.Client(credentials=service_account.Credentials.from_service_account_info(json.loads(open("../credential.json").read()),scopes=["https://www.googleapis.com/auth/bigquery"])))

Let's say that you used Ponder to run some analysis and you want to store the results back to the database. In this tutorial, we will show how you can use the ``to_sql`` command to write your dataframe to table in your database. 

In [2]:
df = pd.read_sql("PONDER.PONDER_CUSTOMER",con=bigquery_con)

After connecting to our `PONDER_CUSTOMER` table, we see that the `C_NAME` column needs to cleaned up.

In [3]:
df.C_NAME = df.C_NAME.str.replace("Customer#",'')

Like in pandas, when you are working with a dataframe, you are always working on a temporary copy of the data. 

By default, all pandas operations returns a copy of the dataframe that was operated on. Ponder never make any modifications to your original data. This is especially important in the data warehouse context as the tables are often regarded as the “source of truth” and can be shared across many teams.

This also means that the reference to your dataframe df is only accessible throughout the session. That means that once you exit the session, the dataframe is no longer accessible. 

So if we plan to use the cleaned up data for our analysis later, we can persist it in a table via `to_sql`.

In [4]:
df

,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,60082,000060082,"x3V6vEbLSeUjYdjS1MvR2,u4gB0S 9d8UEJ",0,10-729-863-1818,3645.47,BUILDING,the accounts. furiously unusual
1,60080,000060080,"g7cKdEj2mzUQLSKFFnWsmL,3GaOIrBmfi",0,10-192-161-6631,689.24,BUILDING,"slyly pending, permanent packages. special fo..."
2,60018,000060018,lQ8PB9FGW53C36XQX2uq0,0,10-310-354-8579,5759.83,BUILDING,ckly bold deposits. carefully bold accounts in...
3,60062,000060062,"1SI,x4F9 zO22 F7OGksMBSUWu5AUpP",0,10-604-525-3386,6210.99,FURNITURE,ons cajole blithely. bold theodolites along
4,60022,000060022,"I2XoZQLC,63R3zIG z6i3VMCS",0,10-513-498-1045,-759.74,FURNITURE,across the blithely ironic sentiments. thinly...
...,...,...,...,...,...,...,...,...
95,60058,000060058,"X9NS,0Ddki",23,33-146-680-6559,6672.12,MACHINERY,ess requests. special requests wake blit
96,60079,000060079,dwwsJWhDr0fnRJnyhe6gtls,24,34-197-192-3607,3329.55,BUILDING,ly special somas poach carefully. furiously un...
97,60059,000060059,"dZISBokE9NWaz13 b5WbOHrd8DifA,e2yict0",24,34-348-323-9173,2337.46,HOUSEHOLD,ndencies. excuses sleep. quickly daring dugout...
98,60033,000060033,fwvb5ua8ZcB,24,34-142-708-2404,-493.59,MACHINERY,lithely final packages. quickly regular reques...


In [5]:
df.to_sql("PONDER.PONDER_CUSTOMER_CLEAN",con=bigquery_con, index=False,if_exists='replace')

Now we can access new new table and continue our analysis whereever we want:

In [6]:
pd.read_sql("PONDER.PONDER_CUSTOMER_CLEAN",con=bigquery_con)

,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,60018,000060018,lQ8PB9FGW53C36XQX2uq0,0,10-310-354-8579,5759.83,BUILDING,ckly bold deposits. carefully bold accounts in...
1,60080,000060080,"g7cKdEj2mzUQLSKFFnWsmL,3GaOIrBmfi",0,10-192-161-6631,689.24,BUILDING,"slyly pending, permanent packages. special fo..."
2,60082,000060082,"x3V6vEbLSeUjYdjS1MvR2,u4gB0S 9d8UEJ",0,10-729-863-1818,3645.47,BUILDING,the accounts. furiously unusual
3,60053,000060053,wb0gteUavl1MPAHwiM,0,10-684-354-3580,6515.51,FURNITURE,ly courts. fluffy packages wake furiously alon...
4,60022,000060022,"I2XoZQLC,63R3zIG z6i3VMCS",0,10-513-498-1045,-759.74,FURNITURE,across the blithely ironic sentiments. thinly...
...,...,...,...,...,...,...,...,...
95,60058,000060058,"X9NS,0Ddki",23,33-146-680-6559,6672.12,MACHINERY,ess requests. special requests wake blit
96,60079,000060079,dwwsJWhDr0fnRJnyhe6gtls,24,34-197-192-3607,3329.55,BUILDING,ly special somas poach carefully. furiously un...
97,60059,000060059,"dZISBokE9NWaz13 b5WbOHrd8DifA,e2yict0",24,34-348-323-9173,2337.46,HOUSEHOLD,ndencies. excuses sleep. quickly daring dugout...
98,60027,000060027,"U,Y EKzklGgxO9HK4uT2g",24,34-250-415-3801,6115.22,MACHINERY,s. ironic packages are carefully slyly unusual...


# Summary

In this tutorial, we learned how to leverage the same pandas API for `pd.to_sql` to save your dataframe to your database. This is often useful if you want to persist the work done on the dataframe beyond your current session. 

In our [next tutorial](https://github.com/ponder-org/ponder-notebooks/blob/main/snowflake/tutorial/05-advanced-sql.ipynb), we will discuss how you can easily move between using Ponder and using SQL when developing your data workflows.